# Loan-To-Value Metric for Lending Protocols

Aidan Price

In [1]:
#Importing Modules

import requests
import json
import pandas as pd
import datetime

In [2]:
#Defining list of lending protocols

protocols = ['compound-v2-ethereum','aave-v3-ethereum','abracadabra-money-ethereum']

In [3]:
# GraphQL Query

query = '''{
  accounts(where: {borrows_: {account_not: null}}) {
    borrows {
      amountUSD
    }
    deposits {
      amountUSD
    }
  }
}
'''

In [4]:
# Defining lists

borrowed_list=[]
deposit_list = []
ltv_list =[]

# Defining ltv function to extract and add borrowed/deposited amounts

def ltv(protocol, protocol_query):
  
  borrows_sum = 0
  deposits_sum = 0

  # Borrows
  for account in protocol_query['borrows']:

    for entry in account:
      borrows_sum += float(entry['amountUSD'])

  # Deposits (Collateral)
  for account in protocol_query['deposits']:

    for entry in account:
      deposits_sum += float(entry['amountUSD'])


  print(f'Amount Borrowed USD: {borrows_sum}')
  print(f'Total Collateral USD: {deposits_sum}')
  LTV = borrows_sum/deposits_sum
  print(f'LTV Ratio: {LTV}')


  if len(borrowed_list) <= 3:
    borrowed_list.append(borrows_sum)
  if len(borrowed_list) <= 3:
    deposit_list.append(deposits_sum)
  
  if len(borrowed_list) <= 3:
    ltv_list.append(LTV)

# ltv(example_data)

In [5]:
for proto in protocols:

  print('--'*10, proto,'--'*10)
  
  # Defining api path
  url_path = f"https://api.thegraph.com/subgraphs/name/messari/{proto}"
  print(url_path)

  # Part 2
  # make the request
  r = requests.post(url_path, json={'query': query})
  # load result into json

  json_data = json.loads(r.text) 

  # Part 3
  data = pd.DataFrame(json_data['data']['accounts'])
  # check result
  # print(data)

  ltv(proto,data)

print(protocols)
print(borrowed_list)
print(deposit_list)
print(ltv_list)

-------------------- compound-v2-ethereum --------------------
https://api.thegraph.com/subgraphs/name/messari/compound-v2-ethereum
Amount Borrowed USD: 42295106.95651911
Total Collateral USD: 69650491.49550605
LTV Ratio: 0.6072477889010741
-------------------- aave-v3-ethereum --------------------
https://api.thegraph.com/subgraphs/name/messari/aave-v3-ethereum
Amount Borrowed USD: 9653886.316913754
Total Collateral USD: 19876409.747374058
LTV Ratio: 0.4856956784254844
-------------------- abracadabra-money-ethereum --------------------
https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-ethereum
Amount Borrowed USD: 22461342.320941545
Total Collateral USD: 33018504.391098075
LTV Ratio: 0.6802652856377472
['compound-v2-ethereum', 'aave-v3-ethereum', 'abracadabra-money-ethereum']
[42295106.95651911, 9653886.316913754, 22461342.320941545]
[69650491.49550605, 19876409.747374058, 33018504.391098075]
[0.6072477889010741, 0.4856956784254844, 0.6802652856377472]


In [7]:
# Change into Dataframe

#To make it look prettier in dashboard
protocol_names = ['Compound V2 Ethereum','Aave V3 Ethereum','Abracadbra Money Ethereum']

ltv_df = pd.DataFrame({
                      'Protocols': protocol_names,
                      'Amount_Borrowed_USD': borrowed_list,
                      'Total_Collateral_USD':deposit_list,
                      'LTV_Ratio':ltv_list 
                       })
ltv_df.head()

# Download as csv
ltv_df.to_csv('Lending_Protocols_ltv.csv', index= False)